## 声明模型

In [40]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

In [41]:
class Base(DeclarativeBase):
    pass

In [42]:
class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [43]:

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

## 创建引擎

In [44]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///database.sqlite", echo=True)
# engine = create_engine("sqlite://", echo=True)
# echo=True参数表示连接发出的 SQL 将记录到标准输出。

In [45]:
# 发出创建表 DDL
Base.metadata.create_all(engine)

2024-12-30 21:47:22,579 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-30 21:47:22,581 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-12-30 21:47:22,581 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-30 21:47:22,582 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-12-30 21:47:22,582 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-30 21:47:22,583 INFO sqlalchemy.engine.Engine COMMIT


In [46]:
## 创建对象并持久化
from sqlalchemy.orm import Session

with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()

2024-12-30 21:47:32,905 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-30 21:47:32,906 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-12-30 21:47:32,908 INFO sqlalchemy.engine.Engine [generated in 0.00007s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('spongebob', 'Spongebob Squarepants')
2024-12-30 21:47:32,909 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-12-30 21:47:32,909 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('sandy', 'Sandy Cheeks')
2024-12-30 21:47:32,910 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-12-30 21:47:32,910 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('patrick', 'Patrick Star')
2024-12-30 21:47:32,911 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?) RETURN

In [47]:
from sqlalchemy import select
session = Session(engine)
stmt = select(User).where(User.name.in_(['spongebob', 'sandy']))
for user in session.scalars(stmt):
    print(user)

2024-12-30 21:48:07,518 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-30 21:48:07,519 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2024-12-30 21:48:07,520 INFO sqlalchemy.engine.Engine [generated in 0.00052s] ('spongebob', 'sandy')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')


In [48]:
# 使用连接进行选择
stmt = (
    select(Address)
    .join(Address.user)
    .where(User.name == 'sandy')
    .where(Address.email_address == 'sandy@sqlalchemy.org')
)
print(stmt)

SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = :name_1 AND address.email_address = :email_address_1


In [50]:
sandy_address = session.scalars(stmt).one()
sandy_address

2024-12-30 21:48:38,221 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2024-12-30 21:48:38,222 INFO sqlalchemy.engine.Engine [cached since 25.54s ago] ('sandy', 'sandy@sqlalchemy.org')


Address(id=2, email_address='sandy@sqlalchemy.org')

In [52]:
# 进行更改
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()
patrick

2024-12-30 21:48:59,730 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-12-30 21:48:59,730 INFO sqlalchemy.engine.Engine [generated in 0.00055s] ('patrick',)


User(id=3, name='patrick', fullname='Patrick Star')

In [53]:
patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))

2024-12-30 21:49:12,147 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-12-30 21:49:12,147 INFO sqlalchemy.engine.Engine [generated in 0.00074s] (3,)


In [54]:
sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"

In [55]:
session.commit()

2024-12-30 21:49:25,664 INFO sqlalchemy.engine.Engine UPDATE address SET email_address=? WHERE address.id = ?
2024-12-30 21:49:25,664 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ('sandy_cheeks@sqlalchemy.org', 2)
2024-12-30 21:49:25,666 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2024-12-30 21:49:25,667 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ('patrickstar@sqlalchemy.org', 3)
2024-12-30 21:49:25,667 INFO sqlalchemy.engine.Engine COMMIT


In [64]:
# 一些删除
sandy = session.get(User, 2)
print(sandy)
print(sandy.addresses)
print(sandy_address)

User(id=2, name='sandy', fullname='Sandy Cheeks')
[Address(id=3, email_address='sandy@squirrelpower.org')]
Address(id=2, email_address='sandy_cheeks@sqlalchemy.org')


In [72]:
# sandy.addresses.remove(sandy_address)

In [67]:
stmt = select(User).where(User.id == 2)
result = session.execute(stmt).scalars().one()
result.addresses

2024-12-30 21:55:23,918 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2024-12-30 21:55:23,919 INFO sqlalchemy.engine.Engine [cached since 20.09s ago] (2,)


[Address(id=3, email_address='sandy@squirrelpower.org')]

In [70]:
from sqlalchemy.orm import joinedload

user = session.query(User).options(joinedload(User.addresses)).filter(User.id == 2).first()
print(user)
print(user.addresses)

2024-12-30 22:00:30,306 INFO sqlalchemy.engine.Engine SELECT anon_1.user_account_id AS anon_1_user_account_id, anon_1.user_account_name AS anon_1_user_account_name, anon_1.user_account_fullname AS anon_1_user_account_fullname, address_1.id AS address_1_id, address_1.email_address AS address_1_email_address, address_1.user_id AS address_1_user_id 
FROM (SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
 LIMIT ? OFFSET ?) AS anon_1 LEFT OUTER JOIN address AS address_1 ON anon_1.user_account_id = address_1.user_id
2024-12-30 22:00:30,306 INFO sqlalchemy.engine.Engine [cached since 48.7s ago] (2, 1, 0)
User(id=2, name='sandy', fullname='Sandy Cheeks')
[Address(id=3, email_address='sandy@squirrelpower.org')]


In [71]:
session.commit()

2024-12-30 22:01:08,493 INFO sqlalchemy.engine.Engine COMMIT


In [73]:
session.delete(patrick)

2024-12-30 22:02:15,991 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-12-30 22:02:15,992 INFO sqlalchemy.engine.Engine [cached since 783.8s ago] (3,)


In [74]:
session.commit()

2024-12-30 22:03:10,273 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2024-12-30 22:03:10,274 INFO sqlalchemy.engine.Engine [cached since 486.4s ago] (4,)
2024-12-30 22:03:10,275 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2024-12-30 22:03:10,276 INFO sqlalchemy.engine.Engine [generated in 0.00052s] (3,)
2024-12-30 22:03:10,276 INFO sqlalchemy.engine.Engine COMMIT
